In [1]:
import sympy as sp

# Define symbols
x, y, z = sp.symbols('x y z', real=True)
vx, vy, vz = sp.symbols('vx vy vz', real=True)
ax, ay, az = sp.symbols('ax ay az', real=True)
phi, theta, psi = sp.symbols('phi theta psi', real=True)
phidot, thetadot, psidot = sp.symbols('phidot thetadot psidot', real=True)
phiddot, thetaddot, psiddot = sp.symbols('phiddot thetaddot psiddot', real=True)
m, g, Ixx, Iyy, Izz = sp.symbols('m g Ixx Iyy Izz', real=True)
K, l, b, Ax, Ay, Az = sp.symbols('K l b Ax Ay Az', real=True)
omega1, omega2, omega3, omega4 = sp.symbols('omega1 omega2 omega3 omega4', real=True)

# Basis vectors
i = sp.Matrix([1, 0, 0])
j = sp.Matrix([0, 1, 0])
k = sp.Matrix([0, 0, 1])

# Rotation matrices
R_x = sp.Matrix([
    [1, 0, 0],
    [0, sp.cos(phi), -sp.sin(phi)],
    [0, sp.sin(phi), sp.cos(phi)]
])

R_y = sp.Matrix([
    [sp.cos(theta), 0, sp.sin(theta)],
    [0, 1, 0],
    [-sp.sin(theta), 0, sp.cos(theta)]
])

R_z = sp.Matrix([
    [sp.cos(psi), -sp.sin(psi), 0],
    [sp.sin(psi), sp.cos(psi), 0],
    [0, 0, 1]
])

R = R_z * R_y * R_x  # Combined rotation matrix

# Angular velocity in body frame
om_b = phidot * i + R_x.T * (thetadot * j) + R_x.T * R_y.T * (psidot * k)

# Inertia tensor
I = sp.Matrix.diag(Ixx, Iyy, Izz)

# Kinetic and potential energy
v = sp.Matrix([vx, vy, vz])
T = 0.5 * m * v.dot(v) + 0.5 * om_b.dot(I * om_b)
V = m * g * z
L = T - V  # Lagrangian

# Generalized coordinates and derivatives
q = sp.Matrix([x, y, z, phi, theta, psi])
qdot = sp.Matrix([vx, vy, vz, phidot, thetadot, psidot])
qddot = sp.Matrix([ax, ay, az, phiddot, thetaddot, psiddot])

# External forces and torques
Thrust = sp.Matrix([0, 0, K * (omega1**2 + omega2**2 + omega3**2 + omega4**2)])
Drag = sp.Matrix([Ax * vx, Ay * vy, Az * vz])
F_ext = R * Thrust - Drag

tau_phi = K * l * (omega4**2 - omega2**2)
tau_theta = K * l * (omega3**2 - omega1**2)
tau_psi = b * (omega1**2 - omega2**2 + omega3**2 - omega4**2)
tau_ext = sp.Matrix([tau_phi, tau_theta, tau_psi])

T_ext = sp.Matrix.vstack(F_ext, tau_ext)

# Equations of motion using Euler-Lagrange method
EOM = sp.Matrix.zeros(6, 1)
for ii in range(6):
    dLdqdot = sp.diff(L, qdot[ii])
    ddt_dLdqdot = sum(sp.diff(dLdqdot, q[j]) * qdot[j] + sp.diff(dLdqdot, qdot[j]) * qddot[j] for j in range(6))
    dLdq = sp.diff(L, q[ii])
    EOM[ii] = ddt_dLdqdot - dLdq - T_ext[ii]

# Substitute Greek symbols for better readability
symbol_subs = {
    sp.Symbol('phi'): 'ϕ',
    sp.Symbol('theta'): 'θ',
    sp.Symbol('psi'): 'ψ',
    sp.Symbol('phidot'): 'ϕ̇',
    sp.Symbol('thetadot'): 'θ̇',
    sp.Symbol('psidot'): 'ψ̇',
    sp.Symbol('phiddot'): 'ϕ̈',
    sp.Symbol('thetaddot'): 'θ̈',
    sp.Symbol('psiddot'): 'ψ̈'
}

# Display the equations of motion without simplification
print("Equations of Motion (Direct Output):")
eom_results = [EOM[i].xreplace(symbol_subs) for i in range(6)]
for i, eom in enumerate(eom_results, 1):
    print(f"EOM[{i}] =")
    print(sp.pretty(eom, use_unicode=True))
    print("\n")




Equations of Motion (Direct Output):
EOM[1] =
                                                 ⎛  2     2     2     2⎞      
Ax⋅vx - K⋅(sin(φ)⋅sin(ψ) + sin(θ)⋅cos(φ)⋅cos(ψ))⋅⎝ω₁  + ω₂  + ω₃  + ω₄ ⎠ + 1.0

     
⋅ax⋅m


EOM[2] =
                                                  ⎛  2     2     2     2⎞     
Ay⋅vy - K⋅(-sin(φ)⋅cos(ψ) + sin(ψ)⋅sin(θ)⋅cos(φ))⋅⎝ω₁  + ω₂  + ω₃  + ω₄ ⎠ + 1.

      
0⋅ay⋅m


EOM[3] =
          ⎛  2     2     2     2⎞                               
Az⋅vz - K⋅⎝ω₁  + ω₂  + ω₃  + ω₄ ⎠⋅cos(φ)⋅cos(θ) + 1.0⋅az⋅m + g⋅m


EOM[4] =
                                                                              
1.0⋅Ixx⋅φ̈ - Ixx⋅ψ̈⋅sin(θ) - Ixx⋅ψ̇⋅θ̇⋅cos(θ) - 0.5⋅Iyy⋅(ψ̇⋅sin(φ)⋅cos(θ) + θ̇

                                                                              
⋅cos(φ))⋅(2⋅ψ̇⋅cos(φ)⋅cos(θ) - 2⋅θ̇⋅sin(φ)) - 0.5⋅Izz⋅(-2⋅ψ̇⋅sin(φ)⋅cos(θ) - 2

                                     ⎛    2     2⎞
⋅θ̇⋅cos(φ))⋅(ψ̇⋅cos(φ)⋅cos(θ) - θ̇⋅sin(φ)) - K⋅l⋅⎝- ω₂  + ω₄ ⎠


EOM[5] =
    